In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import sncosmo
from astropy.cosmology import default_cosmology

project_root = Path.cwd().parent
cosmo = default_cosmology.get()
print(f"Project root: {project_root}")
print(f"Cosmology: {cosmo.name}")

Project root: /Users/david/Code/msc
Cosmology: Planck18


In [2]:
# User input: run folder name 
folder_name = input("Enter the run folder name: ").strip()
run_folder = project_root / "runs" / folder_name

params_path = run_folder / "sncosmo_parameters.csv"

df = pd.read_csv(params_path)
print(f"Loaded {len(df)} object(s) from {params_path}")
df.head()

Enter the run folder name: test2
Loaded 418 object(s) from /Users/david/Code/msc/runs/test2/sncosmo_parameters.csv


,object_id,chisq,ndof,z,t0,x0,x1,c,z_err,t0_err,x0_err,x1_err
0,ZTF17aabtvsy,85.076935,19,0.06922,59867.856566,0.000854,2.999999,0.159425,t0,x0,x1,c
1,ZTF17aabvong,273.362719,27,0.03400,60607.468131,0.003010,0.027239,0.096397,t0,x0,x1,c
2,ZTF17aacldgo,56.901290,13,0.07200,59898.143649,0.000447,-2.999820,0.060099,t0,x0,x1,c
3,ZTF17aadlxmv,60.420066,24,0.06200,58878.408576,0.000929,0.735562,0.157813,t0,x0,x1,c
4,ZTF18aaaonon,109.032639,9,0.07141,59707.246684,0.000593,-2.999805,-0.039216,t0,x0,x1,c


In [3]:
model = sncosmo.Model(source="salt2")
m_B = []
for _, row in df.iterrows():
    model.set(z=row["z"], t0=row["t0"], x0=row["x0"], x1=row["x1"], c=row["c"])
    m_B.append(model.bandmag("bessellb", "ab", 0))
m_B = np.array(m_B)

z_vals = df["z"].values
d_L_Mpc = cosmo.luminosity_distance(z_vals).to_value("Mpc")
mu = cosmo.distmod(z_vals).value

out = df[["object_id", "z", "t0", "x0", "x1", "c"]].copy()
out["m_B"] = m_B
out["mu"] = mu
out["d_L_Mpc"] = d_L_Mpc
out["H0"] = cosmo.H0.to_value("km / (s Mpc)")

# Print out all the results from sncosmo for each object
for idx, row in out.iterrows():
    print("\n=== sncosmo results for object_id:", row["object_id"], "===")
    for col in out.columns:
        print(f"{col}: {row[col]}")

out


=== sncosmo results for object_id: ZTF17aabtvsy ===
object_id: ZTF17aabtvsy
z: 0.06922
t0: 59867.85656580258
x0: 0.0008541163569152
x1: 2.9999989761229267
c: 0.1594247179702288
m_B: inf
mu: 37.54312123332561
d_L_Mpc: 322.57020236370903
H0: 67.66

=== sncosmo results for object_id: ZTF17aabvong ===
object_id: ZTF17aabvong
z: 0.034
t0: 60607.46813112994
x0: 0.0030103433256345
x1: 0.0272392900883518
c: 0.0963974165228478
m_B: inf
mu: 35.94502838417884
d_L_Mpc: 154.52746383053145
H0: 67.66

=== sncosmo results for object_id: ZTF17aacldgo ===
object_id: ZTF17aacldgo
z: 0.072
t0: 59898.143648935176
x0: 0.0004474457035711
x1: -2.999819757141
c: 0.0600990831913313
m_B: inf
mu: 37.63279660284014
d_L_Mpc: 336.1702836306809
H0: 67.66

=== sncosmo results for object_id: ZTF17aadlxmv ===
object_id: ZTF17aadlxmv
z: 0.062
t0: 58878.40857566777
x0: 0.0009285504804469
x1: 0.7355618492557958
c: 0.1578132521170347
m_B: inf
mu: 37.29301060302063
d_L_Mpc: 287.47634900026725
H0: 67.66

=== sncosmo results 

/Users/david/opt/anaconda3/envs/jupyter/lib/python3.11/site-packages/sncosmo/models.py:189: RuntimeWarning: divide by zero encountered in log10
  result[i] = -2.5 * np.log10(f / zpf)


,object_id,z,t0,x0,x1,c,m_B,mu,d_L_Mpc,H0
0,ZTF17aabtvsy,0.06922,59867.856566,0.000854,2.999999,0.159425,inf,37.543121,322.570202,67.66
1,ZTF17aabvong,0.03400,60607.468131,0.003010,0.027239,0.096397,inf,35.945028,154.527464,67.66
2,ZTF17aacldgo,0.07200,59898.143649,0.000447,-2.999820,0.060099,inf,37.632797,336.170284,67.66
3,ZTF17aadlxmv,0.06200,58878.408576,0.000929,0.735562,0.157813,inf,37.293011,287.476349,67.66
4,ZTF18aaaonon,0.07141,59707.246684,0.000593,-2.999805,-0.039216,inf,37.614046,333.279884,67.66
...,...,...,...,...,...,...,...,...,...,...
413,ZTF19aamhhey,0.04500,58567.629684,0.002050,1.029095,0.072741,inf,36.570966,206.154697,67.66
414,ZTF19aamhhgu,0.03820,58568.194219,0.001441,-2.493633,0.078730,inf,36.204578,174.146801,67.66
415,ZTF19aamhhyo,0.08000,58568.014166,0.000563,-0.193397,-0.014214,inf,37.873493,375.576735,67.66
416,ZTF19aamhlnl,0.07600,58566.281588,0.000610,-1.378491,-0.041084,inf,37.756174,355.823635,67.66


In [4]:
# Save to run folder
out_path = run_folder / "distances.csv"
out.to_csv(out_path, index=False)
print(f"Saved to {out_path}")
display(out)

Saved to /Users/david/Code/msc/runs/test2/distances.csv


,object_id,z,t0,x0,x1,c,m_B,mu,d_L_Mpc,H0
0,ZTF17aabtvsy,0.06922,59867.856566,0.000854,2.999999,0.159425,inf,37.543121,322.570202,67.66
1,ZTF17aabvong,0.03400,60607.468131,0.003010,0.027239,0.096397,inf,35.945028,154.527464,67.66
2,ZTF17aacldgo,0.07200,59898.143649,0.000447,-2.999820,0.060099,inf,37.632797,336.170284,67.66
3,ZTF17aadlxmv,0.06200,58878.408576,0.000929,0.735562,0.157813,inf,37.293011,287.476349,67.66
4,ZTF18aaaonon,0.07141,59707.246684,0.000593,-2.999805,-0.039216,inf,37.614046,333.279884,67.66
...,...,...,...,...,...,...,...,...,...,...
413,ZTF19aamhhey,0.04500,58567.629684,0.002050,1.029095,0.072741,inf,36.570966,206.154697,67.66
414,ZTF19aamhhgu,0.03820,58568.194219,0.001441,-2.493633,0.078730,inf,36.204578,174.146801,67.66
415,ZTF19aamhhyo,0.08000,58568.014166,0.000563,-0.193397,-0.014214,inf,37.873493,375.576735,67.66
416,ZTF19aamhlnl,0.07600,58566.281588,0.000610,-1.378491,-0.041084,inf,37.756174,355.823635,67.66
